# Analyse der Bordkarten-kontrolldaten des BERs
Ein Studentenprojekt an der<br>
<img src="https://upload.wikimedia.org/wikipedia/de/thumb/9/90/Hochschule_f%C3%BCr_Wirtschaft_und_Recht_Berlin_logo.svg/500px-Hochschule_f%C3%BCr_Wirtschaft_und_Recht_Berlin_logo.svg.png?20090401201013" width="240" align="left" style="display:block;clear:both;"/>
<br><br><br><br>
Hochschule für Wirtschaft und Recht Berlin<br>
Fachbereich II - Duales Studium Wirtschaft / Technik<br>
<i>Modul: Business Intelligence (WI-201)</i>

<b>Repository</b> (inklusive <b>Voraussetzungen</b> und <b>Installationsanleitung</b>):<br>
https://github.com/italyboy2012/bi-project

## Importe

In [16]:
# Import Datenbank-Konfiguration
#########################################
import dbCredentials
#print(dbCredentials.host)
#print(dbCredentials.username)
#print(dbCredentials.password)

# Import Libs
#########################################
from sqlalchemy import create_engine
import mysql.connector
from mysql.connector import errorcode

#########################################
# Sollten diese nicht installiert sein, dann einmalig folgende Befehle über die Eingabeaufforderung ausführen
    # !pip install mysql-connector-python
    # !pip install SQLAlchemy
    # !pip install PyMySQL
    # (mehr Informationen zum 1. Befehl, MySQL-Connector: https://dev.mysql.com/doc/connector-python/en/connector-python-introduction.html )
    # (mehr Informationen zum 2. Befehl, SQLAlchemy: https://docs.sqlalchemy.org/en/14/index.html )
    
#########################################
from hilfsfunktionen import *
import pandas as pd
from IPython.display import display_html

## Datenbank für Daten aus operativen Quellsystemen anlegen

Ziel dieses Schritts ist es, eine Datenkank für die Daten aus den operativen Quellsystemen (angereichert mit öffentlich zugänglichen Daten) zu erstellen.<br><br>
Stelle sicher, dass der lokale MySQL Server gestartet wurde.

In [17]:
connection = create_server_connection(dbCredentials.host, dbCredentials.username, dbCredentials.password)

db_name = 'biProjekt_Quelldaten'

# Falls bereits eine Datenbank mit dem in der Variable db_name definierten Namen existiert, wird sie mit diesem Befehl gelöscht:
create_database_query = "DROP DATABASE IF EXISTS " + db_name + ";"
create_database(connection, create_database_query)

# Erstellen der Datenbank mit dem in der Variable db_name definierten Namen
create_database_query = "CREATE DATABASE " + db_name + ";"
create_database(connection, create_database_query)


db_connection_str = 'mysql+pymysql://' + dbCredentials.username + ':' + dbCredentials.password + '@' + dbCredentials.host + '/' + db_name
db_connection = create_engine(db_connection_str)

MySQL Database connection successful
Database created successfully
Database created successfully


## Daten der operativen Quellsysteme in die Datenbank laden

Ziel dieses Schritts ist es die Daten aus den operativen Quellsystemen (angereichert mit öffentlich zugänglichen Daten) aus den CSV-Dateien in die im vorherigen Schritt angelegte Datenbank zu laden.

In [18]:
# Daten aus den CSV-Dateien in die DataFrames laden
df_bkk = pd.read_csv("data/bkk-data.csv", sep=";")
df_covid = pd.read_csv("data/covid19_cases.csv", sep=",")
df_airports = pd.read_csv("data/airports.csv", sep=",")
df_gdp = pd.read_csv("data/GDP.csv", sep=",")

# Daten aus den DataFrames in die Datenbank laden
df_bkk.to_sql(con=db_connection, name='BKK', if_exists='replace')
df_covid.to_sql(con=db_connection, name='Covid19', if_exists='replace')
df_airports.to_sql(con=db_connection, name='Airports', if_exists='replace')
df_gdp.to_sql(con=db_connection, name='GDP', if_exists='replace')

## Datentypen setzen

In [19]:
query_table_relations = """
ALTER TABLE `Airports` ADD PRIMARY KEY (`index`);
ALTER TABLE `BKK` ADD PRIMARY KEY (`index`);
ALTER TABLE `GDP` ADD PRIMARY KEY (`index`);

ALTER TABLE `BKK` MODIFY `Destination` VARCHAR(255);
ALTER TABLE `Airports` MODIFY `iata_code` VARCHAR(255);
ALTER TABLE `Airports` MODIFY `municipality` VARCHAR(255);
ALTER TABLE `GDP` MODIFY `Capital City` VARCHAR(255);
"""
# Schlüsselbeziehungen zwischen den Tabellen erstellen
#ALTER TABLE `Airports` ADD FOREIGN KEY (`iata_code`) REFERENCES BKK(`Destination`);
#ALTER TABLE `Airports` ADD CONSTRAINT `BKK Id_fk` FOREIGN KEY (`iata_code`) REFERENCES BKK(`Destination`);
#ALTER TABLE `Faktentabelle_Marketing_Vertrieb` ADD CONSTRAINT `Standortestammdaten Id_fk` FOREIGN KEY (`FilialeID`) REFERENCES `Standortestammdaten`(`FilialeID`);
#ALTER TABLE `Faktentabelle_Marketing_Vertrieb` ADD CONSTRAINT `Produktstamm Id_fk` FOREIGN KEY (`ProduktID`) REFERENCES `Produktstamm`(`ProduktID`);
#ALTER TABLE `Faktentabelle_Marketing_Vertrieb` ADD CONSTRAINT `Werbekampagnenstammdaten Id_fk` FOREIGN KEY (`WerbekampagneID`) REFERENCES `Werbekampagnenstammdaten`(`WerbekampagneID`);

connection = create_db_connection(dbCredentials.host, dbCredentials.username, dbCredentials.password, db_name) # Connect to the Database
execute_query(connection, query_table_relations) 

MySQL Database connection successful
Query successful


## Erstellen des ER-Modells

Ziel dieses Schrittes ist es, aus den eingeladenen Daten ein ER-Modell zu erstellen.

# Einmalig
# !pip install sqlalchemy_schemadisplay
from sqlalchemy import MetaData
from sqlalchemy_schemadisplay import create_schema_graph

db_name = 'biProjekt_Quelldaten'

db_connection_str = 'mysql+pymysql://' + dbCredentials.username + ':' + dbCredentials.password + '@localhost/' + db_name
# create the pydot graph object by autoloading all tables via a bound metadata object
graph = create_schema_graph(metadata=MetaData(db_connection_str),
   show_datatypes=False, # The image would get nasty big if we'd show the datatypes
   show_indexes=False, # ditto for indexes
   rankdir='LR', # From left to right (instead of top to bottom)
   concentrate=False # Don't try to join the relation lines together
)
graph.write_png(db_name+'_schema.png') # write out the file

from IPython.display import Image
Image(filename=db_name+'_schema.png') 

## Erstellen des Sternschemas

In [22]:
# Star Schema:
'''
####################################
PK = Primary Key (Primärschlüssel)
FK = Foreign Key (Fremdschlüssel)
####################################

########Dimensionstabellen:#########

GPSDaten:
        GPSID (PK),
        FlughafenID (FK),
        latitude_deg,
        longitude_deg,
        elevation_ft,
        
Flughafenstammdaten: 
        FlughafenID (PK), 
        Name, 
        Typ,
        IATA,

Zeitdimension: 
        ZeitID (PK), 
        KalMonatJahr, 
        Minute,
        Stunde,
        Tag,
        Monat, 
        Jahr
        
Landesstammdaten:  
        LandID (PK)
        Land,
        Hauptstadt,
        Region
        Subregion,
        Einwohner,
        Dichte,
        Fläche,
        
BIPDimension:
        BIPID (PK)
        LandID (FK)
        BIP (per Capital)
        Wachstumsrate
        WeltAnteil

##########Faktentabelle:#############

Faktentabelle_Flug
        ID (PK),
        LandID(FK von Dimensionstabelle XX),
        FlughafenID (FK),
        BIPID (FK),
        ZeitID (FK),
        Terminal,
        ScanningPosition,
        DocumentType
'''

'\n####################################\nPK = Primary Key (Primärschlüssel)\nFK = Foreign Key (Fremdschlüssel)\n####################################\n\n########Dimensionstabellen:#########\n\nGPSDaten:\n        GPSID (PK),\n        FlughafenID (FK),\n        latitude_deg,\n        longitude_deg,\n        elevation_ft,\n        \nFlughafenstammdaten: \n        FlughafenID (PK), \n        Name, \n        Typ,\n        IATA,\n\nZeitdimension: \n        ZeitID (PK), \n        KalMonatJahr, \n        Minute,\n        Stunde,\n        Tag,\n        Monat, \n        Jahr\n        \nLandesstammdaten:  \n        LandID (PK)\n        Land,\n        Hauptstadt,\n        Region\n        Subregion,\n        Einwohner,\n        Dichte,\n        Fläche,\n        \nBIPDimension:\n        BIPID (PK)\n        LandID (FK)\n        BIP (per Capital)\n        Wachstumsrate\n        WeltAnteil\n\n##########Faktentabelle:#############\n\nFaktentabelle_Flug\n        ID (PK),\n        LandID(FK von Dimensionst

## Erstellen einer OLAP-Datenbank

In [ ]:
df_Standortestammdaten = pd.read_sql('Select f.*, l.LandBez, r.`Region-Bez` From Filialestamm As f'+
                                    ' Join Land As l on f.Land = l.Land'+
                                    ' Join Regionstamm As r on f.Region = r.RegionID'+
                                    ' Order By f.FilialeID;', con=db_connection)

df_Standortestammdaten.set_index("FilialeID", inplace=True)

## Stichprobenartige Auswahl zur Testung des Imports

In [20]:
df_airports.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.947733,-151.692524,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR


In [21]:
df_bkk.head()

,TimeStamp,Terminal,ScanningPosition,Destination,TravelDate,Evaluation,STD,DocumentType
0,01.01.22 04:42,T1,BKK 1.2,NaN,NaN,Rejected,NaN,Unknown
1,01.01.22 04:53,T1,DTC PAV Nord,NaN,NaN,Rejected,NaN,Unknown
2,01.01.22 04:59,T1,BKK 4.4,NaN,NaN,Rejected,NaN,Unknown
3,01.01.22 05:03,T1,BKK 2.3,NaN,NaN,Rejected,NaN,Unknown
4,01.01.22 05:14,T1,BKK 2.4,NaN,NaN,Rejected,NaN,Unknown


In [32]:
df_beispiel = pd.read_sql('Select * FROM BKK'+
                            ' JOIN Airports on BKK.Destination = Airports.iata_code'+
                            ' Order By Airports.iata_code;', con=db_connection)

#df_beispiel.set_index("index", inplace=True)

df_beispiel.head()

,index,TimeStamp,Terminal,ScanningPosition,Destination,TravelDate,Evaluation,STD,DocumentType,index,...,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,48107,03.01.22 11:51,T1,BKK 3.2 Fastlane,AAQ,03.01.22,Rejected,None,PDF417,59632,...,RU,RU-KDA,Anapa,yes,URKA,AAQ,None,http://basel.aero/en/anapa/,https://en.wikipedia.org/wiki/Anapa_Airport,Vityazevo Airport
1,14778,01.01.22 14:40,T1,BKK 4.4,ABA,02.01.22,Rejected,None,PDF417,59570,...,RU,RU-KK,Abakan,yes,UNAA,ABA,None,None,https://en.wikipedia.org/wiki/Abakan_Internati...,None
2,81203,06.01.22 07:00,T1,BKK 2.3,ABJ,06.01.22,Rejected,None,PDF417,20381,...,CI,CI-01,Abidjan,yes,DIAP,ABJ,None,None,https://en.wikipedia.org/wiki/F%C3%A9lix-Houph...,"Abidjan, Port Bouët"
3,66868,04.01.22 17:25,T1,DTC Nord 2,ABV,05.01.22,Rejected,00:00,PDF417,20471,...,NG,NG-FC,Abuja,yes,DNAA,ABV,None,http://www.faannigeria.org/nigeria-airport.php...,https://en.wikipedia.org/wiki/Nnamdi_Azikiwe_I...,None
4,66862,04.01.22 17:24,T1,BKK 2.4,ABV,05.01.22,Rejected,None,PDF417,20471,...,NG,NG-FC,Abuja,yes,DNAA,ABV,None,http://www.faannigeria.org/nigeria-airport.php...,https://en.wikipedia.org/wiki/Nnamdi_Azikiwe_I...,None


## Normalisierung

Die soeben importierten Daten liegen in der nullten Normalform vor.<br>
Ziel dieses Schritts ist es, diese Daten in die dritte Normalform zu überführen.

In [1]:
# code hier


#kann übersprungen werden

## Von OLTP zu OLAP

Ziel dieses Schritts ist es, die nun normalisierten Daten aus dem OLTP-Format in das OLAP-Format zu überführen.<br>
Im konkreten soll ein Stern-Schema erstellt werden, welches die vorhandenen Tabellen in eine Fakten- und in mehrere Dimensions-Tabellen aufsplittet.

In [2]:
# code hier

# Tableau-Tool zum Erstellen eines ETL-Prozesses (einfacher als wir wir es gemacht hätten) --> PREP

## Beantwortung businessrelevanter Fragestellungen

Ziel dieses Schrittes ist es, die nun im Stern-Schema vorliegenden Daten zur Beantwortung von businessrelevanten Fragestellungen zu nutzen.

In [16]:
# code hier

In [31]:
df_ScanningPosition = pd.read_sql("Select `ScanningPosition`, COUNT(`index`) AS `Anzahl BK-Scans` FROM BKK" +
                                " WHERE `Evaluation` LIKE 'Accepted'" +
                                " Group By `ScanningPosition`" +
                                " ORDER BY `Anzahl BK-Scans` DESC;", con=db_connection)

# df = pd.DataFrame({'mass': [0.330, 4.87 , 5.97],
#                   'radius': [2439.7, 6051.8, 6378.1]},
#                  index=['Mercury', 'Venus', 'Earth'])
plot = df_ScanningPosition.plot.pie(y='ScanningPosition', figsize=(5, 5))

TypeError: '<' not supported between instances of 'str' and 'int'